# WSI - Ćwiczenie 2

*Autor: Maksymilian Nowak*

### Cel ćwiczenia

Celem ćwiczenia jest zaimplementowanie algorytmu ewolucyjnego:
- z selekcją turniejową ($k=2$)
- krzyżowaniem jednopunktowym
- mutacją gaussowską
- sukcesją generacyjną

Należy również wykorzystać ten algorytm do znalezienia minimum sumy funkcji $f_1+f_2$, gdzie: 
$$f_1(x_1,y_1)=(x_1^2+y_1-11)^2+(x_1+y_1^2-7)^2$$
$$f_2(x_2,y_2)=2*x_2^2+1.05*x_2^4+\frac{x_2^6}{6}+x_2y_2+y_2^2$$
dla dziedziny $D=[-5,5]\times[-5,5]$, oraz zbadać wpływ dystrybucji populacji początkowej, prawdopodobieństwa mutacji oraz prawdopodobieństwa krzyżowania na współrzędne znalezionych minimów funkcji $f_1$.

## Implementacja algorytmu ewolucyjnego

Do implementacji algorytmu przyjąłem następujące założenia:
- Funkcja celu ma następującą postać: $g(x_1, x_2, y_1, y_2)=f_1(x_1,y_1)+f_2(x_2,y_2)$
- Osobnik jest postaci $(x_1,x_2,y_1,y_2)$
- Kryterium stopu w algorytmie będzie maksymalna liczba pokoleń

Na początku zdefiniuję badane funkcje oraz funkcję celu, będącą ich sumą:

In [1]:
def f1(x, y):
    return (x**2 + y - 11)**2 + (x + y**2 - 7)**2

def f2(x, y):
    return 2*x**2 + 1.05*x**4 + (1/6)*x**6 + x*y + y**2

def g(x1, x2, y1, y2):
    return f1(x1, y1) + f2(x2, y2)

Pozostałymi hiperparametrami algorytmu będą:
- prawdopodobieństwo mutacji
- wariancja używana do obliczania siły mutacji 
- prawdopodobieństwo krzyżowania

Przed zaimplementowaniem algorytmu wyznaczę funkcję oceny - wartością zwracaną przez tę funkcję będzie wynik funkcji celu (im mniejszy, tym lepsza ocena). Dodatkowo kara za wyjście poza dziedzinę funkcji przez osobnika to dodanie 1000 do wyniku działania funkcji oceny.

In [2]:
def rate(genes):
    penalty = 0
    domain_conditions = [-5 <= gene <= 5 for gene in genes]
    if not all(domain_conditions):
        penalty += 1000
    return g(genes[0], genes[1], genes[2], genes[3]) + penalty

Następnie zdefiniuję funkcje odpowiedzialne za reprodukcję, krzyżowanie i mutację, z których będzie się składał algorytm ewoulucyjny.

In [3]:
import random
import numpy as np

# Reprodukcja - selekcja turniejowa (turnieje dwuosobnikowe)
def reproduction(population, rating):
    results = []
    for i, individual in enumerate(population):
        opponents = population.copy()
        opponents.pop(i)
        opponent = random.choice(opponents)
        if rating(individual) < rating(opponent):
            results.append(individual)
        else:
            results.append(opponent)
    return results

# Krzyżowanie jednopunktowe - każda para rodziców produkuje dwójkę dzieci
def crossing(population, crossover_prob):
    results = []
    parents_list = population.copy()
    for i in range(0, len(population) - 1, 2):
        crossover_value = random.uniform(0, 1)
        parent1, parent2 = random.sample(parents_list, 2)
        parents_list.remove(parent1)
        parents_list.remove(parent2)
        if crossover_value < crossover_prob:
            crossing_point = random.randint(1, len(parent1))
            child1, child2 = [
                parent1[:crossing_point] + parent2[crossing_point:],
                parent2[:crossing_point] + parent1[crossing_point:]
            ]
            results.append(child1)
            results.append(child2)
        else:
            results.append(parent1)
            results.append(parent2)
    # Jeśli populacja jest nieparzysta, przenieś rodzica bez pary do wyników
    if len(population) % 2 == 1:
        results.append(parents_list[0])
    return results

# Mutacja gaussowska
def mutation(population, variance, mutation_prob):
    results = []
    for individual in population:
        mutation_value = random.uniform(0, 1)
        if mutation_value < mutation_prob:
            results.append((individual + variance * np.random.normal(0, 1, len(individual))).tolist())
        else:
            results.append(individual)
    return results


#### Zaimplementowany algorytm ewolucyjny

In [4]:
def g_minimum(population, rating, crossover_prob, variance, mutation_prob, max_generations):
    generation = 0
    ratings = [rating(individual) for individual in population]
    best_value = min(ratings)
    best_value_index = ratings.index(best_value)
    best_individual = population[best_value_index]
    while generation < max_generations:
        reproducted = reproduction(population, rating) # Reprodukcja
        crossed = crossing(reproducted, crossover_prob) # Krzyżowanie
        mutated = mutation(crossed, variance, mutation_prob) # Mutowanie
        # Ocena nowego pokolenia
        new_ratings = [rating(individual) for individual in mutated]
        new_best_value = min(new_ratings)
        new_best_value_index = new_ratings.index(new_best_value)
        new_best_individual = mutated[new_best_value_index]
        if new_best_value < best_value:
            best_value = new_best_value
            best_individual = new_best_individual
        # Sukcesja generacyjna
        population = mutated
        ratings = new_ratings
        generation += 1
    return [best_individual, best_value]

### Szukanie rozwiązania

Przy szukaniu rozwiązania będę wykorzystywał zawsze wykorzystywał populację w postaci 20 osobników o identycznych wartościach genów rosnących od -5 do 5.
Poszukiwania rozpocznę od wartości prawdopodobieństw równych 0,5 oraz wariancji równej 0,1.
Maksymalna liczba pokoleń początkowo wyniesie 100.

Ze względu na obecność czynnika losowego przy działaniu algorytmu, dla każdego zestawu hiperparametrów algorytm będzie uruchamiany 20 razy, a następnie otrzymane rezultaty będą uśredniane.

In [10]:
x1 = np.linspace(-5, 5, 20)
x2 = np.linspace(-5, 5, 20)
y1 = np.linspace(-5, 5, 20)
y2 = np.linspace(-5, 5, 20)
individuals = list(map(list, zip(x1, x2, y1, y2)))
print(f"Populacja początkowa: {individuals}")
individual_results = []
value_results = [] 
for i in range(20):
    result = g_minimum(individuals, rate, 0.5, 0.1, 0.5, 100)
    individual_results.append(result[0])
    value_results.append(result[1])
gene_values = list(zip(*individual_results))
average_individual = [
    sum(gene_values[0]) / len(gene_values[0]),
    sum(gene_values[1]) / len(gene_values[1]),
    sum(gene_values[2]) / len(gene_values[2]),
    sum(gene_values[3]) / len(gene_values[3]),
]
average_value = sum(value_results) / len(value_results)
print("Uśrednione wartości genów osobników w postaci [x1, x2, y1, y2]:", average_individual)
print("Średnia wartość funkcji celu:", average_value)

Populacja początkowa: [[-5.0, -5.0, -5.0, -5.0], [-4.473684210526316, -4.473684210526316, -4.473684210526316, -4.473684210526316], [-3.947368421052632, -3.947368421052632, -3.947368421052632, -3.947368421052632], [-3.4210526315789473, -3.4210526315789473, -3.4210526315789473, -3.4210526315789473], [-2.8947368421052633, -2.8947368421052633, -2.8947368421052633, -2.8947368421052633], [-2.368421052631579, -2.368421052631579, -2.368421052631579, -2.368421052631579], [-1.8421052631578947, -1.8421052631578947, -1.8421052631578947, -1.8421052631578947], [-1.3157894736842106, -1.3157894736842106, -1.3157894736842106, -1.3157894736842106], [-0.7894736842105265, -0.7894736842105265, -0.7894736842105265, -0.7894736842105265], [-0.2631578947368425, -0.2631578947368425, -0.2631578947368425, -0.2631578947368425], [0.2631578947368416, 0.2631578947368416, 0.2631578947368416, 0.2631578947368416], [0.7894736842105257, 0.7894736842105257, 0.7894736842105257, 0.7894736842105257], [1.3157894736842106, 1.31

Otrzymane rozwiązanie znajduje się blisko prawdziwego minimum, którego wartość wynosi 0 (minimalną wartość funkcji celu sprawdziłem za pomocą programu WolframAlpha).
Spróbuję teraz zmienić wartość prawdopodobieństwa krzyżowania z 0,5 na 0,8 i powtórzyć eksperyment:

In [6]:
individual_results = []
value_results = [] 
for i in range(20):
    result = g_minimum(individuals, rate, 0.8, 0.1, 0.5, 100)
    individual_results.append(result[0])
    value_results.append(result[1])
gene_values = list(zip(*individual_results))
average_individual = [
    sum(gene_values[0]) / len(gene_values[0]),
    sum(gene_values[1]) / len(gene_values[1]),
    sum(gene_values[2]) / len(gene_values[2]),
    sum(gene_values[3]) / len(gene_values[3]),
]
average_value = sum(value_results) / len(value_results)
print("Uśrednione wartości genów osobników w postaci [x1, x2, y1, y2]:", average_individual)
print("Średnia wartość funkcji celu:", average_value)

Uśrednione wartości genów osobników w postaci [x1, x2, y1, y2]: [2.1550923836642624, 0.007228799763489377, 1.9751937343984447, -0.01821729457777229]
Średnia wartość funkcji celu: 0.008755241698000366


Po zmianie margines błędu algorytmu znacząco zmalał. Teraz zbadam, czy zwiększenie liczby pokoleń wpłynie na otrzymywane wyniki - ustawię wartość tego hiperparametru na 200:

In [7]:
individual_results = []
value_results = [] 
for i in range(20):
    result = g_minimum(individuals, rate, 0.8, 0.1, 0.5, 200)
    individual_results.append(result[0])
    value_results.append(result[1])
gene_values = list(zip(*individual_results))
average_individual = [
    sum(gene_values[0]) / len(gene_values[0]),
    sum(gene_values[1]) / len(gene_values[1]),
    sum(gene_values[2]) / len(gene_values[2]),
    sum(gene_values[3]) / len(gene_values[3]),
]
average_value = sum(value_results) / len(value_results)
print("Uśrednione wartości genów osobników w postaci [x1, x2, y1, y2]:", average_individual)
print("Średnia wartość funkcji celu:", average_value)

Uśrednione wartości genów osobników w postaci [x1, x2, y1, y2]: [2.4474281443489057, -0.012902484961860111, 1.9244329918356065, 0.017149937024210946]
Średnia wartość funkcji celu: 0.003788577108049729


Zgodnie z moimi oczekiwaniami, większa liczba pokoleń wpłynęła pozytywnie na otrzymywany wynik. Otrzymany rezultat uważam za zadowalający, dlatego teraz przejdę do badania wpływu konkretnych parametrów na działanie algorytmu

##### Tabela z otrzymanymi rezultatami

| Prawdopodobieństwo krzyżowania | Wariancja | Prawdopodobieństwo mutacji | Liczba pokoleń | Średni otrzymany wynik |
| --- | --- | --- | --- | --- |
| 0.5 | 0.1 | 0.5 | 100 | 0.014 |
| 0.8 | 0.1 | 0.5 | 100 | 0.009 |
| 0.8 | 0.1 | 0.5 | 200 | 0.004 |

## Badanie wpływu hiperparametrów na algorytm

Podczas przeprowadzania eksperymentów po każdym etapie działania algorytmu ewolucyjnego - reprodukcji, krzyżowania i mutacji - będę generował wykres przedstawiający dystrybucję osobników na planszy - każda funkcja będzie posiadała swój osobny wykres. W tym celu zdefiniuję nową funkcję, która wciąż będzie realizowała algorytm ewolucyjny, ale dodatkowo rysowała wykresy: